In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [24]:
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

In [25]:
df_true.head(10)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
5,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017"
6,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017"
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
8,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017"
9,Alabama official to certify Senator-elect Jone...,WASHINGTON (Reuters) - Alabama Secretary of St...,politicsNews,"December 28, 2017"


In [26]:
df_fake["class"] = 0
df_true["class"] = 1

In [27]:
df_fake.shape, df_true.shape

((23481, 5), (21417, 5))

In [28]:
df_fake_manual_testing = df_fake.tail(50)
for i in range(23480,23430, -1):
    df_fake.drop([i], axis=0, inplace=True)
df_true_manual_testing = df_true.tail(50)
for i in range(21416,21366, -1):
    df_true.drop([i], axis=0, inplace=True)

In [29]:
df_fake.shape, df_true.shape

((23431, 5), (21367, 5))

In [33]:
df_manual_testing = pd.concat([df_fake_manual_testing, df_true_manual_testing], axis=0)
df_manual_testing.to_csv("manual_testing.csv")

In [34]:
df_merge = pd.concat([df_fake, df_true], axis=0)
df_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [35]:
df = df_merge.drop(["title", "date"], axis=1)
df.head(10)

,text,subject,class
0,Donald Trump just couldn t wish all Americans ...,News,0
1,House Intelligence Committee Chairman Devin Nu...,News,0
2,"On Friday, it was revealed that former Milwauk...",News,0
3,"On Christmas day, Donald Trump announced that ...",News,0
4,Pope Francis used his annual Christmas Day mes...,News,0
5,The number of cases of cops brutalizing and ki...,News,0
6,Donald Trump spent a good portion of his day a...,News,0
7,In the wake of yet another court decision that...,News,0
8,Many people have raised the alarm regarding th...,News,0
9,Just when you might have thought we d get a br...,News,0


In [36]:
df = df.sample(frac=1)

In [37]:
df.head(10)

,text,subject,class
20771,Hillary may find out she needs more than black...,left-news,0
13953,A tale of a mother and the dangerous religion ...,politics,0
1392,"Congressman Al Green (D-Texas) thinks, like a ...",News,0
10411,WASHINGTON (Reuters) - The Senate Banking Comm...,politicsNews,1
20905,"According to insider, and author Edward Klein,...",left-news,0
11021,A brave London tour bus passenger saw a Muslim...,politics,0
11277,Things got tense Sunday on 60 Minutes as CBS E...,politics,0
21104,This is a story you will never find in the mai...,left-news,0
8345,BRUSSELS (Reuters) - The European Union’s exec...,politicsNews,1
22014,21st Century Wire says After 7 years of system...,US_News,0


In [38]:
df.isnull().sum()

text       0
subject    0
class      0
dtype: int64

In [39]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [41]:
df["text"] = df["text"].apply(wordopt)

In [42]:
df.head(10)

,text,subject,class
20771,hillary may find out she needs more than black...,left-news,0
13953,a tale of a mother and the dangerous religion ...,politics,0
1392,congressman al green d texas thinks like a ...,News,0
10411,washington reuters the senate banking comm...,politicsNews,1
20905,according to insider and author edward klein ...,left-news,0
11021,a brave london tour bus passenger saw a muslim...,politics,0
11277,things got tense sunday on minutes as cbs eve...,politics,0
21104,this is a story you will never find in the mai...,left-news,0
8345,brussels reuters the european union s exec...,politicsNews,1
22014,century wire says after years of systematic ...,US_News,0


In [43]:
x = df["text"]
y = df["class"]

In [44]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

##### Logistic Regression

In [50]:
from sklearn.linear_model import LogisticRegression

In [51]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [52]:
LR.score(xv_test, y_test)

0.9864285714285714

In [53]:
pred_lr=LR.predict(xv_test)

In [54]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5863
           1       0.98      0.99      0.99      5337

    accuracy                           0.99     11200
   macro avg       0.99      0.99      0.99     11200
weighted avg       0.99      0.99      0.99     11200



##### Decision Tree

In [55]:
from sklearn.tree import DecisionTreeClassifier

In [56]:
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [57]:
pred_dt = DT.predict(xv_test)

In [58]:
DT.score(xv_test, y_test)

0.9963392857142858

In [59]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5863
           1       1.00      1.00      1.00      5337

    accuracy                           1.00     11200
   macro avg       1.00      1.00      1.00     11200
weighted avg       1.00      1.00      1.00     11200



##### Gradient Boosting

In [60]:
from sklearn.ensemble import GradientBoostingClassifier

In [61]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [62]:
pred_gbc = GBC.predict(xv_test)

In [63]:
GBC.score(xv_test, y_test)

0.9959821428571428

In [64]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5863
           1       0.99      1.00      1.00      5337

    accuracy                           1.00     11200
   macro avg       1.00      1.00      1.00     11200
weighted avg       1.00      1.00      1.00     11200



##### Random Forest Classifier 

In [65]:
from sklearn.ensemble import RandomForestClassifier

In [66]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [67]:
pred_rfc = RFC.predict(xv_test)

In [68]:
RFC.score(xv_test, y_test)

0.9892857142857143

In [69]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5863
           1       0.99      0.99      0.99      5337

    accuracy                           0.99     11200
   macro avg       0.99      0.99      0.99     11200
weighted avg       0.99      0.99      0.99     11200

